# Setup and run ML Service experiment
This notebook will setup and run an Azure Machine Learning Service experiment

In [1]:
import azureml.core
from azureml.core import Experiment, Workspace

print("This notebook was created using version 1.0.48 of the Azure ML SDK")
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")

This notebook was created using version 1.0.48 of the Azure ML SDK
You are currently using version 1.0.69 of the Azure ML SDK


In [2]:
script_folder = "./experiment"

In [3]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True, verbose=True)

subscription_id = os.environ.get('subscription_id')
resource_group = os.environ.get('resource_group')
workspace_name = os.environ.get('workspace_name')

tenant_id = os.environ.get('tenant_id')
service_principal_id = os.environ.get('service_principal_id')
service_principal_password = os.environ.get('service_principal_password')

C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\dotenv\main.py:52: UserWarning: File doesn't exist 
  warnings.warn("File doesn't exist {}".format(self.dotenv_path))  # type: ignore


In [47]:
try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    ws.write_config()
    print('Library configuration succeeded')
except:
    print('Workspace not found')

ERROR - 627afb82-e26b-40e3-add3-0d67d029a46b - OAuth2Client:Get Token request failed
Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\util\connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connectionpool.py", line 597, in urlopen
    self._prepare_proxy(conn)
  File "C:\Appl\applications\Mi

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


ERROR - e0f305fe-dc43-4f4f-b49b-093eadcc31a9 - OAuth2Client:Get Token request failed
Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\util\connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connectionpool.py", line 597, in urlopen
    self._prepare_proxy(conn)
  File "C:\Appl\applications\Mi

Workspace not found


Access the workspace using the provided configuration file. You need to download this from the portal if you haven't setup with the step above.

In [ ]:
ws=Workspace.from_config()

ERROR - 7818177b-120b-4d38-ac47-bea76718f289 - OAuth2Client:Get Token request failed
Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\azureml\core\authentication.py", line 1359, in _get_arm_token_with_refresh
    auth, _, _ = profile_object.get_login_credentials(resource)
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\azureml\_vendor\azure_cli_core\_profile.py", line 509, in get_login_credentials
    account = self.get_subscription(subscription_id)
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\azureml\_vendor\azure_cli_core\_profile.py", line 471, in get_subscription
    raise AuthenticationException("Please run 'az login' to setup account.")
azureml.exceptions._azureml_exception.AuthenticationException: AuthenticationException:
	Message: Please run 'az login' to setup account.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
   

Performing interactive authentication. Please follow the instructions on the terminal.


Turn on diagnostics collection

In [50]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


Create / access the experiment

In [ ]:
experiment = Experiment(workspace=ws, name='ml-service-test')

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


Create / access compute

In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "mltest-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 2)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")
vm_priority = os.environ.get("AML_COMPUTE_CLUSTER_PRIORITY", "dedicated") # "lowpriority",

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it. ' + compute_name)
else:
    print('creating a new compute target...')  
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                           min_nodes=compute_min_nodes,
                                                           max_nodes=compute_max_nodes,
                                                           vm_priority=vm_priority,
                                                           # vnet_resourcegroup_name='ddpdRGDev',
                                                           # vnet_name='azureml-vnet',
                                                           # subnet_name='azureml-subnet',
                                                          )
    
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)
    
    compute_target.wait_for_completion(show_output=True)

Creating new cluster
Creating
Succeeded
AmlCompute wait for completion finished
Minimum number of nodes requested have been provisioned


In [ ]:
Setup an estimator - we will use the SKLearn specific estimator.

In [27]:
from azureml.train.sklearn import SKLearn

script_params = {
    '--kernel': 'linear',
    '--penalty': 1.0,
}

estimator = SKLearn(source_directory=script_folder, 
                    script_params=script_params,
                    compute_target=compute_target,
                    entry_script='train.py',
                    pip_packages=['joblib', 'matplotlib']
                   )

WARNING - You have specified to install packages in your run. Note that you have overridden Azure ML's installation of the following packages: ['joblib']. We cannot guarantee image build will succeed.


In [43]:
run = experiment.submit(estimator)

ERROR - e9121327-f0f5-4f38-be2b-2759ed77fdf8 - OAuth2Client:Get Token request failed
Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\util\connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connectionpool.py", line 597, in urlopen
    self._prepare_proxy(conn)
  File "C:\Appl\applications\Mi

Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
ERROR - 7e95301f-a011-428c-9e96-c3e1b760a0e6 - OAuth2Client:Get Token request failed
Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\connection.py", line 160, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\site-packages\urllib3\util\connection.py", line 57, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Appl\applications\Miniconda3\envs\azure\lib\socket.py", line 745, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Appl\applications\Miniconda3\envs\azu

TrainingException: TrainingException:
	Message: Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /common/oauth2/token (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000202534ED828>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',)))
	InnerException AuthenticationException:
	Message: Please ensure you have network connection. Error detail: HTTPSConnectionPool(host='login.microsoftonline.com', port=443): Max retries exceeded with url: /common/oauth2/token (Caused by ProxyError('Cannot connect to proxy.', NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000202534ED828>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',)))
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
   

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()

In [42]:
run.wait_for_completion(show_output=True)

RunId: ml-service-test_1571994322_57a700d2
Web View: https://mlworkspace.azure.ai/portal/subscriptions/d4e5fecf-32d0-4314-a56e-ca2389ac7ac3/resourceGroups/DataPlatformMHEWRGDev/providers/Microsoft.MachineLearningServices/workspaces/mhew-ml-service-workspace/experiments/ml-service-test/runs/ml-service-test_1571994322_57a700d2

Streaming azureml-logs/55_azureml-execution-tvmps_94c7f6ff43aa315bf27d56abbec5b861dbf7a049d1cefe87b89dab4ee5c88b73_d.txt

2019-10-25T09:08:52Z Starting output-watcher...
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_56ecd87888ebba66c1d19f08b9aa70b0
a1298f4ce990: Pulling fs layer
04a3282d9c4b: Pulling fs layer
9b0d3db6dc03: Pulling fs layer
8269c605f3f1: Pulling fs layer
6504d449e70c: Pulling fs layer
4e38f320d0d4: Pulling fs layer
b0a763e8ee03: Pulling fs layer
11917a028ca4: Pulling fs layer
a6c378d11cbf: Pulling fs layer
6cc007ad9140: Pulling fs layer
6c1698a608f3: Pulling fs layer
9576438f0736: Pulling fs layer
bc33b7d6f79c: Pull

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with SyntaxError: invalid syntax (train.py, line 73)",
        "details": [],
        "debugInfo": {
            "type": "SyntaxError",
            "message": "invalid syntax (train.py, line 73)",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/mhew-ml-service-workspace/azureml/ml-service-test_1571994322_57a700d2/mounts/workspaceblobstore/azureml/ml-service-test_1571994322_57a700d2/azureml-setup/context_manager_injector.py\", line 99, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/azureml-envs/azureml_81c27c3a95fa31c28e2cd92c280c98a1/lib/python3.6/runpy.py\", line 261, in run_path\n    code, fname = _get_code_from_file(run_name, path_name)\n  File \"/azureml-envs/azureml_81c27c3a95fa31c28e2cd92c280c98a1/lib/python3.6/runpy.py\", line 236, in _get_

In [ ]:
print(run.get_metrics())

In [ ]:
print(run.get_file_names())

In [ ]:
run.properties['azureml.git.repository_uri']
run.properties['azureml.git.branch']
run.properties['azureml.git.commit']
run.properties['azureml.git.dirty']

In [ ]:
# Get the best run
minimum_rmse_runid = None
minimum_rmse = None

for run in experiment.get_runs():
    run_metrics = run.get_metrics()
    run_details = run.get_details()
    # each logged metric becomes a key in this returned dict
    run_rmse = run_metrics["Accuracy"]
    run_id = run_details["runId"]

    if minimum_rmse is None:
        minimum_rmse = run_rmse
        minimum_rmse_runid = run_id
    else:
        if run_rmse < minimum_rmse:
            minimum_rmse = run_rmse
            minimum_rmse_runid = run_id

print("Best run_id: " + minimum_rmse_runid)
print("Best run_id rmse: " + str(minimum_rmse))

In [ ]:
from azureml.core import Run
best_run = Run(experiment=experiment, run_id=minimum_rmse_runid)
print(best_run.get_file_names())

In [ ]:
best_run.download_file(name="model_alpha_0.1.pkl")

In [9]:
envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML"):
        print("Name",env)
        print("packages", envs[env].python.conda_dependencies.serialize_to_string())

Name AzureML-Tutorial
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.0.69
  - azureml-defaults==1.0.69
  - azureml-telemetry==1.0.69
  - azureml-train-restclients-hyperdrive==1.0.69
  - azureml-train-core==1.0.69
  - azureml-widgets==1.0.69
  - azureml-pipeline-core==1.0.69
  - azureml-pipeline-steps==1.0.69
  - azureml-opendatasets==1.0.69
  - azureml-automl-core==1.0.69
  - azureml-train-automl==1.0.69
  - azureml-explain-model==1.0.69
  - azureml-tensorboard==1.0.69
  - azureml-mlflow==1.0.69
  - mlflow
  - sklearn-pandas
- pandas
- numpy
- tqdm
- scikit-learn
- matplotlib
name: azureml_15549395a113679a9fb7f144ae9386ff

Name AzureML-Minimal
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.0.69
  - azureml-defaults==1.0.69
name: azureml_5f6ffffd4487e99555beea9f6150391b

Name AzureML-Chainer-5.1.0-GPU
packages channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.0.69
  - azurem